<a href="https://colab.research.google.com/github/ujs/MachineLearningCode/blob/master/perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
 #Import libraries
 from random import seed
 from random import randrange
 from csv import reader
 import numpy as np

In [10]:
a = np.array([1,2])
a = a.astype('float')
a.dtype

dtype('float64')

In [0]:
#load data
def load_csv(filename):
  dataset = list()
  f = open(filename, 'r')
  for row in f.readlines():
    if not row:
      continue
    else:
      dataset.append(row)
    dataset = np.array(dataset)
    return dataset


In [0]:
#preprocess data

def convert_to_float(dataset,column):
  dataset[:,column] = dataset[:,column].astype('float')
  return dataset

#convert categorical labels into integer
def convert_to_int(dataset, column): 
  unique = set(list(dataset[:,column]))
  lookup = dict()
  for i, value in enumerate(unique):
    lookup[value] = i
  dataset[]



In [0]:
#Split data into train, validation, test

In [0]:
#Perceptron algorithm implementation (training)

In [0]:
#Cross Validation

In [0]:
#Evaluation of algorithm

In [0]:
#Plotting results